<a href="https://colab.research.google.com/github/YMGYM/TSE_Learning/blob/master/air_pollution_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

First, I used the data about air_polution data from 
Ministry of Environment

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

I got the tip from how to read the file with utf-8 encoding from [here](https://teddylee777.github.io/pandas/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%9C%EA%B8%80%EA%B9%A8%EC%A7%90%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0%EB%B0%A9%EB%B2%95)

In [ ]:
def read_data():
  data = pd.read_csv("/content/drive/My Drive/Datasets/air_2019.csv", encoding='utf-8')
  return data

In [ ]:
data = read_data()

In [ ]:
data

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥)
0,20191231,100,도심권,111121,중구,26,18,0.023,0.012,0.4,0.002
1,20191231,100,도심권,111123,종로구,27,19,0.021,0.015,0.4,0.003
2,20191231,100,도심권,111131,용산구,14,15,0.020,0.015,0.4,0.003
3,20191231,101,서북권,111181,은평구,24,13,0.024,0.010,0.4,0.003
4,20191231,101,서북권,111191,서대문구,23,9,0.023,0.013,0.5,0.002
...,...,...,...,...,...,...,...,...,...,...,...
8891,20190101,103,서남권,111301,양천구,46,29,0.014,0.033,0.6,0.002
8892,20190101,104,동남권,111261,강남구,37,31,0.012,0.028,0.6,0.005
8893,20190101,104,동남권,111262,서초구,37,20,0.015,0.032,0.4,0.004
8894,20190101,104,동남권,111273,송파구,44,28,0.011,0.042,0.8,0.004


Filtering data

In [ ]:
location = data['측정소명'] == "중구"

dust_location = data[location]
dust_location['미세먼지(㎍/㎥)']

0       26
25      36
50      46
75      43
100     34
        ..
8771    63
8796    55
8821    35
8846    31
8871    38
Name: 미세먼지(㎍/㎥), Length: 356, dtype: int64